Selecting File

In [1]:
# https://stackoverflow.com/questions/3579568/choosing-a-file-in-python-with-simple-dialog

from tkinter import Tk     # from tkinter import Tk for Python 3.x
from tkinter.filedialog import askopenfilename

Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file
print(filename)

C:/Users/paste/OneDrive/바탕 화면/CSmatS/B-side/adv_protocol_two.log


Open content

In [2]:
# from my Advent of Code 2024 codes

# 파일을 읽기 모드로 열기
file = open(filename, "r")

# 파일 내용 읽기
content = file.read()

# 파일 닫기
file.close()

# 읽은 내용 출력
print(content)

content_list_str = list(map(str, content.split("\n")))

while content_list_str[-1] == '': # remove unintended line break
    content_list_str = content_list_str[0: -1] # wow recursive programming moment

content_list = []

for i in range(len(content_list_str)):
    content_list.append(dict(eval(content_list_str[i]))) # I knew this would be much better with json library at this point

print(content_list)
#len(content_list)

{"opcode": 0, "type": "RSA"}
{"opcode": 1, "type": "RSA", "public": 41669, "parameter": {"n": 194477}}
{"opcode": 2, "type": "RSA", "encrypted_key": [79145, 119027, 1981, 90654, 133728, 170717, 9040, 7654, 175504, 61889, 99610, 161410, 62799, 12557, 170726, 117589, 52317, 116597, 61889, 107515, 99468, 168759, 159597, 177359, 144894, 91329, 173097, 97103, 110315, 68256, 180874, 140635]}
{"opcode": 2, "type": "AES", "encryption": "F1PwMZ387D31dJ0fyCmZBA=="}
[{'opcode': 0, 'type': 'RSA'}, {'opcode': 1, 'type': 'RSA', 'public': 41669, 'parameter': {'n': 194477}}, {'opcode': 2, 'type': 'RSA', 'encrypted_key': [79145, 119027, 1981, 90654, 133728, 170717, 9040, 7654, 175504, 61889, 99610, 161410, 62799, 12557, 170726, 117589, 52317, 116597, 61889, 107515, 99468, 168759, 159597, 177359, 144894, 91329, 173097, 97103, 110315, 68256, 180874, 140635]}, {'opcode': 2, 'type': 'AES', 'encryption': 'F1PwMZ387D31dJ0fyCmZBA=='}]


Checks whether given json follows the example format

In [3]:
def line_validity_check(target_list: list, line_number: int, opcode: int, typ: str, additional_keys: list = []): # type is a function so could not use exact word
    error_list = []
    target = target_list[line_number]
    
    keys = target.keys()
    
    if "opcode" in keys:
        if target["opcode"] != opcode:
            error_list.append(f"unpredicted opcode (!= {opcode})")
    else:
        error_list.append("missing opcode")
    if "type" in keys:
        if target["type"] != typ:
            error_list.append(f"unpredicted type (!= {typ})")
    else:
        error_list.append("missing type")
    
    for key in additional_keys:
        if key not in keys:
            error_list.append(f"missing {key}")
    
    # no need to handle when keys not mentioned in additional_keys are in target ig
    
    return error_list

In [4]:
def json_validity_check(target: list):
    if len(target) < 4:
        print("incomplete json that does not contain enough information to decode messages")
        return False
    
    l0 = line_validity_check(target, 0, 0, "RSA")
    l1 = line_validity_check(target, 1, 1, "RSA", ["public", "parameter"])
    l2 = line_validity_check(target, 2, 2, "RSA", ["encrypted_key"])
    for i in range(3, len(target)): # handling multiple messages
        exec(f"l{i} = line_validity_check(target, {i}, 2, \"AES\", [\"encryption\"])")
    
    turnagain = True
    for i in range(len(target)):
        current = eval(f"l{i}")
        if current != []:
            print(f"error in line {i}\n{target[i]}\n{str(current)[1:-1].replace('\'', '')}") # removing bracket and single quote (why did I waste time here)
            turnagain = False
    return turnagain # re-turn haha I'm funny...

In [5]:
json_validity_check(content_list)

True

Find key values

In [6]:
# written by parksh5295

def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

In [7]:
def find_pq(n: int, p_min: int = 400, p_max: int = 500) -> list:
    for p in range(p_min, p_max + 1):
        if is_prime(p):
            if n % p == 0:
                q = int(n / p)
                if is_prime(q):
                    return [p, q] # few of handlings seems unnessary, but just in case
    print("could not find p and q in given range")
    return False

In [8]:
def is_relative_prime(a: int, b: int):
    '''RSA: checks whether [a] and [b] are relatively prime to each other
    returns True when [a] and [b] are relative prime, else return False
    requires function [is_prime]'''
    
    if a <= 0 or b <= 0:
        print('Issue from [is_relative_prime] function: more than one value is not positive')
        return False

    high = max(a, b)
    low = min(a, b)
    
    if low == 1 or high == low:
        return True
    
    for i in range(2, low + 1):
        if is_prime(i):
            if low % i == 0 and high % i == 0:
                return False
            
    return True

def d_from(e: int, phin: int):
    '''RSA: calculates inverse of [e] in modular of [phin]
    checks whether [e] and [phin] are relative prime before calculating, therefore requiring function [is_relative_prime]'''
    if not is_relative_prime(e, phin):
        print("Issue from [d_from] function: e and phin are not relative prime")
        return -1
    for i in range(1, phin + 1): # planed to optimize this part with EEA, but cancelled
        if e * i % phin == 1:
            return i
    return -2 #this should never happen

In [9]:
n = dict(content_list[1]["parameter"])[list(dict(content_list[1]["parameter"]).keys())[0]] # yeah I love few unreasonably long line when coding

p, q = find_pq(n)

e = content_list[1]["public"]

d = d_from(e, (p - 1)*(q - 1))

if not is_relative_prime(e, (p - 1)*(q - 1)):
    print("somehow e and phi_n is not relative prime")

Decryptes key list

In [10]:
def rsa_decryption(encyrpted_message, d, n):
    '''RSA: decrypes given [encyrpted_message] with using private key pair ([d], [n])'''
    return encyrpted_message**d % n

In [11]:
encrypkeyed = content_list[2]["encrypted_key"]

# print(len(encrypkeyed))

decrypted_key_list = []

for i in encrypkeyed:
    decrypted_key_list.append(rsa_decryption(i, d, n))
    
decrypted_key = bytes(decrypted_key_list)
print(decrypted_key)

b'pF\xa9\x07v\x1c\xfc\xe1}\xb9&af\xc9\x84l\x81O\xb9\xae\x98\xddB\xb3\x15\xa3r\x0cn\xb0\x97\xca'


Finding the message

In [12]:
# from example

from Crypto.Cipher import AES
import base64

def decrypt_aes(key, encrypted):
    """Decrypt a message using AES-ECB."""
    aes = AES.new(key, AES.MODE_ECB)
    decrypted = aes.decrypt(base64.b64decode(encrypted))
    return decrypted.rstrip(decrypted[-1:]).decode()

In [ ]:
for i in range(3, len(content_list)):
    print(decrypt_aes(decrypted_key, content_list[i]["encryption"])) # I was looking forward for some more dramatic message

mathematics
